# **Multiple View Geometry in Computer Vision**

This is my notes while working through the Multiple View Geometry in Computer Vision course on Youtube by Sean Mullery.

## **Lecture 1: Basic Image Processing**

Image Processing is a task that changes an image in some way for the sake of a human viewer. In many cases, it is the first pre-processing step in a computer vision pipeline.

The simplest image processing operators are point operators that manipulate each pixel independently of the others like *brightness* or *contrast* adjustment and color correction.

### **Color Systems**

If we have an image in the RGB color palette, we cannot just treat the colors as independent if we just want to increase the brightness. The issue is that color channels are highly correlated from each other and so they cannot be treated independently. Instead we can change from RGB to some other channel that allows us to access the brightness and modify it.


Different light sources have different color distributions. For example, sunlight is quite blue in tone, while fluorescent, LED, camera flash etc. all have different associated colors. So we might want to *balance* the color of an image taken under such conditions. In many cases there are competing light sources in an image and a single correction is not globally effective. When switching from one color system to another, we generally multiply the color vector by an associated matrix. 

### **Compositing and Matting**

Matting refers to extracting an object from an image, while compositing refers to inserting an artifact into an image. We add an $\alpha$ channel that denotes how much the pixel is transparent/opaque. We want $\alpha = 1$ for the foreground and $\alpha = 0$ for the background. Then we composit using the equation $C = (1-\alpha) B + \alpha F$ where $B$ is the background image and $F$ the foreground image.

### **Linear Filters**

Blurring is an example of a linear filter. We blur an image to trade off sharpness in an image to reduce noise. One example is a box or averaging filter. 




The Horizontal and vertical Sobel filters $$\begin{bmatrix}1 & -2 & -1 \\ 0 & 0 & 0 \\ 1 & 2 & 1 \end{bmatrix}\,\,\,\, \text{and} \,\,\,\,\begin{bmatrix}-1 & 0 & 1 \\ -2 & 0 & 2 \\ -1 & 0 & 1 \end{bmatrix}$$ look for corresponding edges in an image.

The Laplacian filter is an approximation of $\nabla^2 B = \frac{\partial^2 B}{\partial x^2} + \frac{\partial^2 B}{\partial y^2}$. A common choice is $\begin{bmatrix}-1 & -1 & -1 \\ -1 & 8 & -1 \\ -1 & -1 & -1\end{bmatrix}$

### **Non-linear Operators**

Not all noise will have a Gaussian distribution. One example is shot noise where some noise is much outside the range of the standard deviation of the pixel values. A regular gaussian filter would be overpowered by such noise, but a median filter would be much more preferable for this task. However a median filter is not as efficient at eliminating Gaussian noise.  

### **The Bilateral Filter**

None of the above filters deal well with an edge in the image that is part of the signal. What occurs is that such edges are smoothed by the pixels close to them so we get a softer edge (lower frequency). Bilateral filters reject pixels whose intensity differs too much from the central pixel. So we smooth/average only by those pixels that are close in value to the central pixel. Unlike previous filters, Bilateral filters depend on the intensity values of the input image and not just on the location of pixels.

The formula is $$g(i,j) = \frac{\sum_{k,l} f(k,l) w(i,j,k,l)}{\sum_{k,l} w(i,j,k,l)}$$ The weighting coefficient $w(i,j,k,l)$ is complex and needs to be broken down in to constituent parts. It is the product of a domain kernel $$d(i,j,k,l) = \exp \left(- \frac{(i-k)^2 + (j-l)^2}{2 \sigma_d^2}\right)$$ and a range kernel $$r(i,j,k,l) = exp\left(- \frac{\lvert\lvert f(i,j) - f(k,l)\rvert\rvert^2}{2 \sigma_r^2}\right)$$

## **Lecture 2: Edge Detection and Features**

Edges in images convey an enormous amount of information, and a line drawing of an image contains most of the information needed to understand it. Edges are related to changes in the brightness of pixels, and so gradients will invariably turn up in their study. 

### **Edge Detection**

Some important edge detection algorithms include:

* **Sobel and Prewitt Filters**. These filters use a Gaussian filter to smooth the image before looking for edges, even though this may seem a bit counterintuitive. Then they compute partial derivatives in $x$ and $y$ derivatives, and threshold the magnitude of the gradient.
* **Marr-Hildreth**. Here they compute the laplacian after gaussian smoothing and look for *zero crossings* along each row and column. At a zero crossing $a, -b$, we compute $\lvert a + b\rvert$ and threshold this value to separate the true edges. Since we have $\nabla^2 (g * I) = (\nabla^2 g) * I$, we just have to pre-compute $\nabla^2 g$ and then use it as a filter.
* **Canny Edge Detector**. Canny wanted the edge detection system to be accurate (correct number of predicted edges), precise (edges identified in the right place), and robust to noise. The image is, again, first smoothed with a gaussian filter, and then the gradient (magnitude and orientation) is computed. Then *non-maximum suppression* is applied to choose the best guess for the location of the gradient (and no repeated guesses very close to a guessed edge). And after that, a *hysteresis threshold* is applied.

Let us look at the steps taken by the Canny edge detector in more detail.

1. We combine the smoothing and derivative computations by convolving with the derivative-of-gaussian filters $\nabla S = \nabla (g * I) = (\nabla g) * I$. Then we compute the magnitude of the gradient $\sqrt{S_x^2 + S_y^2}$, as well as the orientation $\theta = \arctan\left(\frac{S_y}{S_x}\right)$. 
2. We now perform *non-maximum suppression*. Since we know the direction of the gradient, we can take a slice along the direction perpendicular to the gradient (and hence the edge). Along each such slice, we determine the pixel with the largest gradient magnitude (or one larger than both neighbors). We would like each of the picked points to form a line/curve. This allows the edge to be more sharp.
3. Finally, we perform a *hysteresis threshold*. Here we set a high threshold and a low threshold. Then if a pixel is above a high threshold, we include all of its neighbors unless they drop below the low threshold. We continue this until there are no neighbors left, or we dip below the low threshold. After this, we scan the image from left-right and top-bottom. If the gradient at a pixel is above the high threshold, it is declared an edge pixel. If the gradient is below the low threshold, it is declared a non-edge. For those pixels between the thresholds, we consider the neighbors iteratively and then mark it an edge pixel if it is connected to an edge pixel directly via pixels that are not below the low threshold. 

### **Simple Features**

In computer vision, we are looking for "features" in images. We want to find features that will be easy to identify (not vague) even in a transformed image.  This means that the feature description should be somewhat invariant to the imaging angle and brightness. One application of feature matching is matching features so that we can stitch images to form a panorama. Features that easy to localize are called *key-point features* or *interest points* and usually the patch of pixels surrounding the point location are used to describe them. 

There are two broad approaches for this:

1. Find points of interest in one image and then search for each of these in the other image. This works well for small baseline (i.e when the distance betwen image view points is small). This might be consecutive frames from a video sequence where only small changes have occurred.
2. Find points of interest separately in both images, and then try to match the points of images. This is useful in large baseline problems like stitching panoramas, recognizing objects from different views, etc.

What makes a good feature?

* Background items like a blue sky or plain walls are poor features since they look the same everywhere.
* Edges are also not optimal since all movement parallel to the edge look similar. Further both end points might look similar.
* Corners are better. These are points with gradients in at least two significantly different orientations, and they are easy to pin point.
* When looking for good features, a useful technique is that of auto-correlation. Here we test a patch of image against a small region around it and if it is very different from them, then we are satisfied. So we are looking to maximize $$E_{AC}(\Delta \vec{u}) = \sum_{i} w(\vec{x}_i)  \left[I_0(\vec{x}_i + \Delta \vec{u}) - I_0(\vec{x}_i)\right]^2$$ where $\vec{u} = (u,v)$ is the displacement vector. We compute a weighted sum over each of the pixels $\vec{x}_i$ in the patch of interest.

The auto-correlation algorithm above is an expensive operation, and this led to much research to find ways to calculate good approximations of the autocorrelation surface. Lucas, Kanade and Shi, Tomasi use the first parts of a Taylor series expansion of $I_0(\vec{x}_i + \nabla \vec{u}) \approx I_0(\vec{x}_i) + \nabla I_0(\vec{x}_i) \cdot \nabla \vec{u}$. Putting this back in the above equation, we get $$E_{AC}(\Delta \vec{u})\approx \sum_{i} w(\vec{x_i}) \left[\nabla I_0(\vec{x_i}) \cdot \Delta \vec{u}\right]^2 = \sum_{i} w(\vec{x_i}) \vec{u}^T I_0(\vec{x}_i)I_0(\vec{x}_i)^T\vec{u} = \sum_{i} w(\vec{x}_i) \Delta \vec{u}^T A_i \vec{u}$$ These symmetric matrices can be decomposed by Spectral theorem and we get $A = U\begin{bmatrix}\lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix}U^T$. These eigenvectors and eigenvalues represent the directions of greatest and least change. If we find great change in both directions, then there is enough reason to pick this as a point of interest.

 ### **Finding Simple Shapes**

 Many computer vision applications require us to look for specific shapes. We may be looking to detect lane markings or road signs, for example. Starting with the idea of a line, we image that we have a number of points and wish to determine the best straight line to fit these points.

**RANSAC (Random Assignment and Consensus)**

The RANSAC algorithms solves the *outlier problem*. It is a good method if we know which points apply to which line. We run the following a fixed number of times.

1. Randomly select two points to fit an (exact) line.
2. Solve the model parameters for just these points.
3. Then look at how many of the other points are consistent with this model (within some margin of the line). The number of points is considered the number of "votes" or a measure of consensus for this model.

This way outliers have no effect as they will not have any votes. The RANSAC algorithm is much more computationally efficient than the hough transform. Also, this algorithm can work with any parametric curve representation.

**The Hough Transform**

This allows us to identify which points are consistent with which shape. We look at the parameter space and carve out the curve corresponding to the parameters which lead to curves passing through a given point. Then intersecting curves in this parameter space correspond to fitting curves in the image.

## **Lecture 3. Geometric Transformations**



Similarities preserve angles. Affine transformations preserve parallel lines, while projective transformations preserve straight lines. The corresponding matrices are:

1. For rigid motions $$\begin{bmatrix}\mathbf{R} & \mathbf{t} \\ \mathbf{0}^{T} & 1\end{bmatrix} = \begin{bmatrix}\cos \theta & -\sin \theta & t_x \\ \sin\theta & \cos\theta & t_y \\ 0 & 0& 1 \end{bmatrix}$$
2. For similarities  $$\begin{bmatrix}s\mathbf{R} & \mathbf{t} \\ \mathbf{0}^{T} & 1\end{bmatrix} = \begin{bmatrix}s\cos \theta & -s\sin \theta & t_x \\ ss\sin\theta & \cos\theta & t_y \\ 0 & 0& 1 \end{bmatrix}$$
3. For affine transformations $$\begin{bmatrix}\mathbf{A} & \mathbf{t} \\ \mathbf{0}^{T} & 1\end{bmatrix} = \begin{bmatrix}a_{11} & a_{12} & t_x \\ a_{21} & a_{22} & t_y \\ 0 & 0& 1 \end{bmatrix}$$ using SVD, an affine transformation can be broken down into $A = \mathbf{R}(\theta)\mathbf{R}(-\phi)\mathbf{D}\mathbf{R}(\phi)$ where $\mathbf{D} = \begin{bmatrix} \sigma_1 & 0 \\ 0 &\sigma_2 \end{bmatrix}$
4. For projective transformations we have an arbitrary transformation $$H = \begin{bmatrix}h_{11} & h_{12} & h_{13} \\ h_{21} & h_{22} & h_{23} \\ h_{31} & h_{32}& h_{33} \end{bmatrix}$$ 


### **3D Vectors**

When we get to camera views, we will need to consider relative movement. To describe how a 3D point would change under 3D transformations, we will use homogeneous coordinates. Our matrices will now have shape $3 \times 4$ or $4 \times 4$. Projective transformations are important because they relate the 3D coordinates to a point of view, known as the center of projection. 


2D rotations are relatively straightforward, but 3D rotations are not. We can try to separate the rotations into rotation around the three axes separately $$\begin{align*} R_{\alpha}^x &= \begin{bmatrix} 1 & 0  & 0 \\ 0 & \cos \alpha & -\sin \alpha \\ 0& \sin \alpha & \cos\alpha \end{bmatrix} \\ R_{\beta}^y &= \begin{bmatrix} \cos\beta & 0  & \sin \beta \\ 0 & 1 & 0 \\ -\sin\beta & 0 & \cos\beta \end{bmatrix} \\ R_{\gamma}^z &= \begin{bmatrix} \cos\gamma & -\sin\gamma & 0 \\ \sin\gamma & \cos \gamma & 0 \\ 0& 0 & 1 \end{bmatrix} \\ \end{align*}$$ Note that $R_\alpha^x R_\beta^y \neq R_\beta^y R_\alpha^x$. Here $\alpha,\beta,\gamma$ are called the *Euler angles*, but these parameters are not smooth functions of the rotations. So small rotations can lead to big changes in the Euler angles. This can refer to an issue known as *gimbal lock*. The issue is that rotations do not form a group under addition.

However, Euler angles can still be useful. For example, if we have simple known rotations, perhaps between two fixed camera positions along one axis, Euler angles can be useful.

### **Infinitesimal 3D rotations**

We would like to determine a smooth path between two views rather than just determining the full rotations that will lead from one to another. 


If $A$ is a skew-symmetric matrix then all the eigenvalues of $A$ are zero or purely imaginary. Also, there is an orthogonal matrix $V$ such that $A = V \Sigma V^T$ where $\Sigma = \text{diag}(A_1,\ldots, A_m, 0,\ldots, 0)$ where $A_i = \begin{bmatrix} 0 & a_i \\ -a_i &0 \end{bmatrix}$ (this can be proved by using the spectral theorem and noting that if $A(x + iy) = ia (x + iy)$ then $A(y + ix) = -ia(y + ix)$). In computer vision, a common skew-symmetric matrix is given by $$u_x = \begin{bmatrix}0 & -u_3 & u_2 \\ u_3 & 0 & -u_1 \\ -u_2 & u_1 & 0 \end{bmatrix}$$ This is a linear operator from $\mathbb{R}^{3}$ to the space of skew-symmetric 3x3 matrices. It can be easily checked that $u_x \cdot v = u \times v$ for $v \in \mathbb{R}^3$. Thus, if $u \neq 0$, we have $\text{rank}(u_x) = 2$ and the kernel of $u_x$ is spanned by $u$.

#### **Lie Groups and Lie Algebras**

For a matrix $A$ we define $\exp(A) = I + A + \frac{A^2}{2} + \ldots = \sum_{n=0}^{\infty} \frac{A^n}{n!}$. For example, if $A = \begin{bmatrix} 0 & -\theta \\ \theta & 0 \end{bmatrix}$ then $\exp(A) = \begin{bmatrix} \cos \theta  & -\sin \theta \\ \sin \theta & \cos \theta \end{bmatrix}$. More generally, if we have a skew symmetric matrix $B$, then $\exp(B)$ will be an element of the special orthogonal group $O(n)$. This allows us to parameterize the orthogonal group in a manner that works well with additive transformations of the parameter.


The power series $\exp(A)$ converges absolutely and uniformly, along with partial derivatives on any bounded set. The absolute convergence means that we can reorder terms, and the uniform convergence means that we  can differentiate term by term. This means that $\frac{d}{dt} \exp(t A) = A \exp(tA)$. If $AB = BA$ then $\exp(A)\exp(B) = \exp(A+B)$. In particular, $\exp(-A) = \exp(A)^{-1}$

The exponential map has a local inverse $\text{Log} : U \to V$ where $U \subseteq \mathfrak{gl}(n,\mathbb{R})$ is a neighborhood of the origin and $V \subset \text{GL}(n,\mathbb{R})$ is a neighborhood of the identity matrix. We have $\text{Log}(I + X) = X - \frac{X^2}{2} + \frac{X^3}{3} + \ldots$.

In general, for $A$,$B$ sufficiently small, $\text{Log}(\exp A\exp B)= \text{Log}\left(I + A + B + AB + \frac{A^2}{2} + \frac{B^2}{2} + \frac{AB^2}{2} + \ldots\right) = \log(I +X)$. So we have $$\text{Log}(\exp A \exp B) = (A+B)  + \frac{AB - BA}{2} + \ldots$$ We write $[A,B] = AB -  BA$. This is called the *commutator*. Even the higher terms can be written in terms of commutators, by the *Baker-Campbell-Hausdorff formula*. For example, the cubic term is $\frac{1}{12}[A, [A,B]] - \frac{1}{12}[B,[A,B]]$

So we have $\exp(A)\exp(B) = \exp(A + B + [A,B] + \ldots)$ for $A$ and $B$.

Let $G$ be a topologically closed subgroup of $\text{GL}(n,\mathbb{R})$. Let $$\mathfrak{g} = \left\{X \in \mathfrak{gl}(n,\mathbb{R}) : \exp(tX) \in G \text{ for all } t \in \mathbb{R}\right\}$$. Then $\mathfrak{g}$ is a vector space and closed under the commutator $[\cdot,\cdot]$. Also, $\mathfrak{g}$ is parallel to the tangent space of $G$ at $I$. And $\exp : \mathfrak{g} \to G$ is locally invertible.

Topologically closed subgroups of $\text{GL}(n,\mathbb{R})$ are Lie groups. There are examples of lie groups that aren't matrix groups, but most interesting examples are so.

Consider the matrix group $O(n)$. Then the Lie algebra is $\mathfrak{o}(n) = \{X : \exp(tX)^T\exp(tX) = I\text{ for all } t \in \mathbb{R}\}$. But $\exp(tX)^T\exp(tX) = \exp(tX^T)\exp(tX) = I$. This means $\exp(tX^T) = \exp(-tX)$. For small enough $t$, we have tX^T = -tX so that $X^T = -X$. So $\mathfrak{o}(n)$ is contained in the space of skew-symmetric matrices. The converse is straightforward.

$\mathfrak{g}$ is the tangent space to the identity of $G$. Indeed, suppose that $\gamma$ is a path in $G$ such that $\gamma(0) = I$. Then $\gamma'(0) \in \mathfrak{g}$. For this we need to show that $\exp(t \gamma'(0)) \in G$ for all $t$. For this, it suffices to show that $\exp(\gamma'(0)) \in G$ since we may then work with the path $s \to \gamma(st)$ instead.



Now for small $s$ define $h(s) = \text{Log}(\gamma(s))$. We have $h'(0) = D_{1}\text{Log}(\gamma'(0)) = (D_0\exp)^{-1}\gamma'(0) = \gamma'(0)$. 


But $h'(0) = \lim_{\varepsilon \to 0} \frac{h(\varepsilon) - h(0)}{\varepsilon} = \lim_{\varepsilon \to 0}\frac{h(\varepsilon)}{\varepsilon}$. Take $\varepsilon = \frac{1}{n}$ for $n \in \mathbb{N}$. This means $nh(1/n) \to \gamma'(0)$. But $\gamma(s) = \exp h(s) \in G$ for all $s$, so $\gamma(s)^n = \exp(n h(s)) \in G$ for $s$ small. This means $\exp(n h(1/n)) \in G$ for all $n$, and so $\exp(\gamma'(0)) \in G$.

#### **Lie Groups and Lie Algebras in Computer Vision**

Consider a family of rotation matrices $R(t)$ which continuously transforms a point from its original location $R(0) = I$ to a different one.  We have $X_t = R(t)X_0$. Now $R(t)^TR(t) = I$ so by differentiating, we get that $R'(t)R^T$ is skew-symmetric. Thus, $R'(t)R^T = w_x(t)$ and so $R'(t) = w_x(t)R(t)$. Since $R(0) = I$, $R'(0) = w_x(0)$. So the skew symmetric matrix $w_\times(0)$ gives the first order approximation of a rotation $$R(dt) = R(0) + dR = I + wx(0)dt$$ We have $R(t) = \exp(w_\times t)$ and we can use Rodrigue's formula to get $R(t) = I + w_\times \sin(t) + w_\times^2(1 - \cos t)$.

## **Lecture 4: Camera Models**

Figures in the real world undergo a projective transformation when captured by a camera.

In general, we must go from world coordinates (3D) to camera coordinates (3D) to image coordinates (2D) to pixel coordinates (2D discrete). 

1. The first transformation to camera coordinates is a rigid body motion based on the extrinsic parameters of the camera (location, orientation).
2. Next we must go from the camera coordinates to the image coordinates. We will simply drop one dimension and divide the first two coordinates by the third and multiply by the focal length.
3. Finally, we will change to pixel coordinates from image coordinates. This depends on the *intrinsic* parameters of the camera. 

So we have $$\lambda \begin{bmatrix} x \\y \\ 1\end{bmatrix} = \begin{bmatrix} s_x & 0 & O_x \\ 0 & s_y & O_y \\ 0 & 0 &1 \end{bmatrix} \begin{bmatrix} f & 0 & 0 \\ 0 & f & 0 \\ 0 & 0 &1 \end{bmatrix}\begin{bmatrix} 1 & 0 & 0 & 0\\ 0 & 1 & 1 & 0 \\ 0 & 0 &1 & 0 \end{bmatrix}\begin{bmatrix} \mathbf{R} & \mathbf{t} \\ \mathbf{0}^T & 1 \end{bmatrix} \begin{bmatrix} X \\ Y \\ Z \\ 1 \end{bmatrix} $$ where $\lambda$ is the nonlinear scaling factor by the third coordinate in the second step above.

The intrinsic parameters are often put together into one matrix called $K = \begin{bmatrix} fs_x & 0 & O_x \\ 0 & fs_y & O_y \\ 0 & 0 & 1\end{bmatrix}$

We will want to work backwards to reconstruct the 3-dimensional information from the image. So we will want to attempt to identify $M$ and then decompose it as above. This is called *Structure in Motion*. 

Lenses differ from an ideal pinhole camera model. One issue is *radial distortion* where straight lines in the real world are curved. This can be corrected using rectilinear methods and dense methods (using a known grid in the real world).

## **Lecture 5: Photometry to Geometry**

When we want to re-construct the 3D world, we will start with images which contain brightness or color values. This makes feature recognition in an image important. Here we refer to primitive features such as edges, corners, or lines, rather than composite objects like cars (the task of recognizing such objects is called *object recognition*).

We cannot determine 3D information from a single image, so we will need to find corresponding points in more than one image even if they are taken under different brightness conditions or perspectives. So we will need to recognize features in two images.

Reconstruction from a small number of points is called sparse reconstruction, while reconstruction with many points is called dense reconstruction.

If we have two camera views and we know that cameras were close together (low baseline), then we only need to search a small distance in the image. But small baselines make 3D reconstruction more difficult.  

The small deformation model usually originates from images generated by the same camera. This means that the views are separated in time as well, but for small baseline, we assume that there is a small displacement in space and time. So even with video (30-60fps) this limits the speed of movement as the gap between frames could be large. In the autonomous vehicle setting the camera or other cars can move quickly, though they do at least tend to move smoothly and in predictable directions.

### **Optical Flow Estimation**

For small displacement, classical optic flow estimation can be used. Lucas-Kanade and Horn-Schunck are methods dating back to 1981. The former is a sparse method, but the latter is a dense method given displacement for each pixel in the image. Optic Flow refers to *apparent flow*, the flow in a perspective image. This may or may not correspond to actual movement in the real world.

We will make some assumption in optical flow. We assume that the scene between views is static (scene doesn't change even if the views might), the movements are rigid (no deformation), and that brightness stays constant. The brightness constancy assumption can be defined as follows. Let $x(t)$ denote a moving point at time $t$, and $I(x,t)$ a sequence of images (video). Then, we assume that $I(x(t), t)$ is constant. So we would want $$ \frac{d}{dt} I(x(t), t) = (\nabla I (t))^T \left(\frac{dx}{dt}\right) + \frac{\partial I}{\partial t} = 0 $$ 

What we are looking to identify is how pixels have moved in this image, that is, $v = \frac{dx}{dt}$. But we cannot compute this directly and we can only solve for the component in the direction of the gradient. However, if we make a further assumption that there is constant motion $v$ in some small neighborhood of pixels, then we can try to find a best vector $v$ that solves the equation. 

We will call the pixels in this small window neighborhood of $x$ $W(x)$. Then we should have $$ \nabla I(x', t)^T v + \frac{\partial I}{\partial t}(x',t) = 0 $$ for all $x' \in W(x)$. So we will minimize $$\ell(v) = \sum_{W(x)} \lvert \nabla I(x',t)^T v + I_t(x',t)\rvert^2 dx'$$ Taking derivatives, we have $$\frac{\partial \ell}{\partial v} = \sum_{W(x)} (2 \nabla I \cdot \nabla I^T v + 2I_t \nabla I)dx' = 0$$ Let us define $$M = \sum_{W(x)}\begin{bmatrix} I_x^2 & I_xI_y \\ I_xI_y & I_y^2 \end{bmatrix}dx'$$ and $q = \sum_{W(x)} I_t \nabla I \cdot dx'$. Then we have $2 M v + 2q = 0$. If $M$ is invertible, we can find $v = -M^{-1}q$. We will have to handle the case where $M$ is not invertible separately.

As we are in the image plane, we can move in $x$ and $y$ directions only, and so the velocity vector will be two dimensional. Using the velocity vector $v$ above, we can track points over frames.

If the matrix $M$ is not invertible, Lucas-Kanade can fail entirely. If the matrix $M$ is not invertible, the matrix cannot be recovered. One possible cause could be if the window has a constant intensity (for example the flat side of a surface). A constant intensity in the spatial domain means $\nabla I(x) = 0$ and $I_t(x) $ for all points in the window.

So a simple feature tracking algorithm is as follows:

* For a given time sample $t$, compute at each coordinate $(x,y)$ in the image frame the matrix $M = \sum_{W(x)} \begin{bmatrix}I_x^2 & I_xI_y\\ I_xI_y & I_y^2 \end{bmatrix}$.
* Mark all coordinates for which the determinant of $M$ is larger than a threshold $\theta > 0$, i.e $\lvert M(x)\rvert \geq \theta$.
* For all those coordinates, the local velocity vector can be calculated by $v(x,t) = -M(x)^{-1} \begin{bmatrix} \sum_{W(x)} I_xI_t dx' \\ \sum_{W(x)} I_yI_tdx'\end{bmatrix}$
* Repeat the above steps for the points $x + v$ at time $t + 1$.


## **Lecture 6: Epipolar Geometry and Two View Reconstruction**

To solve the two view reconstruction problem, we make the following assumptions:

* We assume that we are viewing a static scene and that we have two different views of this scene.
* We will assume that we already have a set of point correspondences in our two views.
* We assume that we know the intrinsic parameters of the camera (focal length, width, height of pixels etc.) and that these are the same for both views.

Even with these assumptions, we don't know the extrinsic parameters (3D coordinate positions of the cameras), and we don't know the 3D coordinates that correspond to the points in the image. The route to a solution is as follows:

* We need to disentangle the 3D coordinates from the camera motion (rotation/translation) algebraically.
* Remove the 3D coordinates from the algebra so that we have equations in 2D image coordinates only.
* Use these to solve for camera motion (using the 8-point algorithm).
* Now we have the extrinsic parameters we can determine the 3D coordinates.

We will call a point in the 3D world $X$, and the projections to the two views $x_1$ and $x_2$. We refer to the center of the projection of the two cameras as $O_1$ and $O_2$. 

### **Epipolar Geometry**

If we draw a line between the two centres of projection, this line will intersect the two image planes. The points of intersection are called the *epipoles* $e_1$ for the intersection with the first view image plane and $e_2$ for the intersection with the second view image plane. The *epipolar plane* is the plane defined by the camera centers and world point, while the *epipolar lines* are the lines from the points $x_i$ to the corresponding epipoles $e_i$.


Firstly, let's deal with the intrinsic parameters. We will assume no skew in the pixels and assume there is a 1:1 aspect ratio. We can also assume that the focal length is $1$, and that the origin is the center of the camera frame. This means that the intrinsic parameter matrix is $$K = \begin{bmatrix} fs_x & 0 & O_x \\ 0 & fs_y & O_y \\ 0 & 0 & 1 \end{bmatrix} = \begin{bmatrix} 1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 1 \end{bmatrix}$$ 

As a result, we have $\lambda x_1 = X$. For the second camera, we must describe the coordinates in terms of camera 1. So we will have $\lambda_2 x_2 = RX + T= \lambda_1 R x_1 + T$. We can move $T$ from one side of the equation to the other by multiplying by $T_\times$. Then we will get $\lambda_2 T_\times x_2 = \lambda_1 T_\times R x_1$. Now we have $0 = \lambda_1 x_2^T T_\times R x_1$, so we have $x_2^T T_\times Rx_1 = 0$. This is called the *epipolar constraint*. It relates the 2D constraints without the 3D coordinates.

Let $E = T_\times \mathbb{R} \in \mathbb{R}^{3\times 3}$. This is called the *essential matrix* and has rank 2. The epipolar constraint stipulates that the triangle $\vec{O_1X}, \vec{O_2O_1}, $ and $\vec{O_2X}$ lies on a plane. Such a plane should have a volume of $0$, so we have a relation like $x_2^T (T_\times R) x_1 = 0$. 

Since $E$ has rank 2, it is a singular matrix. Thus, we cannot obtain the two $\lambda$ values from just the epipolar constraint. We will need eight sets of corresponding points, and hence this is called the 8-point algorithm. If we find enough point correspondences, we should be able to recover $E$. We would like to identify $R$ and $T$ from $E$. 

The space of essential matrices is $\mathcal{E} = \left\{T_\times R : R \in SO(3), T \in \mathbb{R}^3\right\} \subset \mathbb{R}^{3 \times 3}$. This is the tangent bundle of $SO(3)$. A non-zero matrix $E \in \mathbb{R}^{3 \times 3}$ is an essential matrix iff E = U\Sigma V^T$ with $\Sigma = \begin{bmatrix} \sigma & 0 & 0 \\ 0 & \sigma &0 \\ 0 & 0 & 0 \end{bmatrix}$ and $U,V \in SO(3)$. Even given the essential matrix, there are two possible decompositions $(T,R)$. But usually, only one of them will make sense since the depths will be negative for the other one. 

To obtain the essential matrix, we will start with the 3x3 matrix we obtain from our linear methods and then project that on to the space of essential matrices. This is an easy option, but trades off accuracy. For this we will use the **Eight Point Algorithm**. 

We will start with the epipolar constraint $x_2^T E x_1 = 0$. This should hold for any matching point in the two views. If we have enough points, we will use eight points corresponding points and then we should be able to recover the unknown matrix. This assumes that the eight points satisfy some criteria and have no noise. 

Let $E = [e_{ij}]$ be the essential matrix. We will write the elements of $E$ as a column vector $e$. Given a pair of point correspondences $(x,y,1)$ and $(x',y',1)$, define the vector $v = \begin{bmatrix} x'x & x'y &y'x &y'y & y' & x' & x & y & 1 \end{bmatrix}$. Each such point correspondence will give a constraint $v \cdot e = 0$. We will need 8 of them to obtain the matrix $E$ up to scaling.

To project this matrix onto the essential space, we will change the singular values $\sigma_1, \sigma_2, \sigma_3$ to $\sigma, \sigma, 0$ where $\sigma = \frac{\sigma_1 + \sigma_2}{2}$.

### **Two View Reconstruction**

If we know $E$, we can get $R$ and $T$ up to a scale factor. To take the scale factor into account, we assume $\lvert\lvert E\rvert \rvert = \lvert\lvert T \rvert\rvert = \gamma$, a currently undetermined value. We will get $\lambda_1^j x_{2\times}^j Rx_1^k + \gamma x_{2x}^j T = 0$. This can be written as a linear system in variables $\lambda_1^j$ and $\gamma$. Again, we will find enough constraints from a sufficient number of points. 

If the camera views are a known distance apart, we can calculate the scale of the objects. If we don't know the intrinsic matrix, we will extend the essential matrix and look at the *fundamental matrix*.

If there is no translation, recovery of 3D geometry is not possible in this configuration.